# Use ChatOllama

- TODO - Automate installation of Ollama for local devbox setup

In [1]:
model_name = "llama3.2:latest"

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
    # other params...
)

from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Je aime le programmation.', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-26T05:31:19.643641Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1720113417, 'load_duration': 559992459, 'prompt_eval_count': 45, 'prompt_eval_duration': 833000000, 'eval_count': 7, 'eval_duration': 54000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-88153d63-1894-48f6-9f07-6db8eb9e5858-0', usage_metadata={'input_tokens': 45, 'output_tokens': 7, 'total_tokens': 52})

## Chaining

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe Programmierung.', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-26T05:31:23.086124Z', 'done': True, 'done_reason': 'stop', 'total_duration': 217513750, 'load_duration': 32372375, 'prompt_eval_count': 40, 'prompt_eval_duration': 139000000, 'eval_count': 6, 'eval_duration': 44000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-db95ef2e-d415-4850-9ce7-420a28c1e724-0', usage_metadata={'input_tokens': 40, 'output_tokens': 6, 'total_tokens': 46})

## Tool Calling

In [4]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model=model_name,
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

[{'name': 'validate_user',
  'args': {'addresses': ['123 Fake St', '234 Pretend Boulevard'],
   'user_id': 123},
  'id': '5384b9fe-29b8-4697-ace6-7d20c67f9546',
  'type': 'tool_call'}]

### Debug tool calling by introduing DebuggableChatOllama

In [5]:
class DebuggableChatOllama:
    def __init__(self, llm):
        """
        Initialize the DebuggableChatOllama proxy for debugging interactions with the LLM.
        """
        self.llm = llm
        self.bound_tools = None

    def bind_tools(self, tools):
        """
        Bind tools to the LLM and log the tools being bound.
        """
        print("\n--- Debug: Binding Tools ---")
        for tool in tools:
            print(f"Binding Tool: {tool.__name__ if hasattr(tool, '__name__') else str(tool)}")
        print("--- End Debug ---\n")
        self.bound_tools = self.llm.bind_tools(tools)
        return self

    def __call__(self, prompt):
        """
        Log and invoke the prompt on the bound LLM.
        """
        if self.bound_tools is None:
            raise RuntimeError("Tools have not been bound. Call bind_tools first.")
        
        print("\n--- Debug: Sending Prompt ---")
        print(prompt)
        print("--- End Debug ---\n")
        
        response = self.bound_tools.invoke(prompt)
        
        print("\n--- Debug: LLM Response ---")
        print(response)
        print("--- End Debug ---\n")
        
        return response

# Example usage
if __name__ == "__main__":
    model_name = "llama3.3:latest"  # Replace with your model name
    base_llm = ChatOllama(
        model=model_name,
        temperature=0,
    )

    debug_llm = DebuggableChatOllama(base_llm)
    debug_llm = debug_llm.bind_tools([validate_user])

    prompt = "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
    response = debug_llm(prompt)
    print(f"Final Response: {response}")



--- Debug: Binding Tools ---
Binding Tool: name='validate_user' description='Validate user using historical addresses.\n\n    Args:\n        user_id (int): the user ID.\n        addresses (List[str]): Previous addresses as a list of strings.' args_schema=<class 'langchain_core.utils.pydantic.validate_user'> func=<function validate_user at 0x137cbfb00>
--- End Debug ---


--- Debug: Sending Prompt ---
Could you validate user 123? They previously lived at 
--- End Debug ---


--- Debug: LLM Response ---
content='' additional_kwargs={} response_metadata={'model': 'llama3.3:latest', 'created_at': '2025-01-26T05:31:40.642743Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9562956542, 'load_duration': 810959083, 'prompt_eval_count': 209, 'prompt_eval_duration': 4337000000, 'eval_count': 36, 'eval_duration': 4412000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-6baaedc5-fb54-4717-b756-02deb4e37585-0' tool_calls=[{'name': 'validate_user'

## More tool calling

### Python functions

In [6]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]
llm = ChatOllama(
    model=model_name,
    temperature=0,
)
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.3:latest', 'created_at': '2025-01-26T05:31:51.545194Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5304960417, 'load_duration': 30338500, 'prompt_eval_count': 228, 'prompt_eval_duration': 2641000000, 'eval_count': 22, 'eval_duration': 2632000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-587a499e-6a2a-4872-a549-3bf9adae3714-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '148ea80e-207c-45d1-8563-4e1d1db46df4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 228, 'output_tokens': 22, 'total_tokens': 250})

### Pydantic class

You can bind tools using Pydantic classes as well

In [7]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]
llm = ChatOllama(
    model=model_name,
    temperature=0,
)
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.3:latest', 'created_at': '2025-01-26T05:31:58.080368Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3739856417, 'load_duration': 23681917, 'prompt_eval_count': 228, 'prompt_eval_duration': 1105000000, 'eval_count': 22, 'eval_duration': 2610000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-96808d02-aed6-45e1-a937-b7818e2c9799-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '2b52ad0d-3543-4787-9b71-eea4497eb4a5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 228, 'output_tokens': 22, 'total_tokens': 250})

### TypedDict class

You can bind tools using TypedDicts and annotations

In [8]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]
llm = ChatOllama(
    model=model_name,
    temperature=0,
)
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.3:latest', 'created_at': '2025-01-26T05:32:03.843935Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3856594708, 'load_duration': 28243958, 'prompt_eval_count': 228, 'prompt_eval_duration': 1202000000, 'eval_count': 22, 'eval_duration': 2625000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-2821ab2e-8b96-4e1c-8dc3-4154348d8dad-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '5cd8cf6f-400a-480c-9e5a-837bffe73d10', 'type': 'tool_call'}], usage_metadata={'input_tokens': 228, 'output_tokens': 22, 'total_tokens': 250})

In [9]:
query = "What is 3 * 12? Also, what is 11 + 49?"

result = llm_with_tools.invoke(query)

result.tool_calls


[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'da29abea-b957-45b0-a03e-171a83c3b0fe',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': '61039664-545e-4fee-a2bd-ef20e71af5e1',
  'type': 'tool_call'}]

In [10]:
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage, ToolMessage

# Define tool functions
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

# Define tools
tools = [
    Tool(name="add", description="Add two numbers.", func=add),
    Tool(name="multiply", description="Multiply two numbers.", func=multiply),
]

# Initialize the LLM
model_name = "llama3.2:latest"  # Replace with your actual model name
llm = ChatOllama(
    model=model_name,
    temperature=0,
)
llm_with_tools = llm.bind_tools(tools)

# Define the query and wrap it in a HumanMessage
query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(content=query, role="human")]

# Invoke the LLM with tools
ai_msg = llm_with_tools.invoke(messages)

# Output the tool calls made by the LLM
print(ai_msg.tool_calls)

# Append the AI's response to the messages
messages.append(ai_msg)

# Process tool calls
for tool_call in ai_msg.tool_calls:
    # Extract tool name and arguments
    tool_name = tool_call["name"].lower()
    tool_args = tool_call["args"]
    tool_call_id = tool_call["id"]  # Extract the tool_call_id

    # Map `__arg1` and `__arg2` to `a` and `b`
    mapped_args = {
        "a": tool_args.get("__arg1"),
        "b": tool_args.get("__arg2"),
    }

    try:
        # Call the tool with the mapped arguments
        selected_tool = {"add": add, "multiply": multiply}[tool_name]
        result = selected_tool(**mapped_args)
    except Exception as e:
        result = f"Error processing tool call '{tool_name}': {e}"

    # Create a tool message to append to messages
    tool_msg = ToolMessage(
        tool=tool_name,
        content=str(result),  # Convert result to string for compatibility
        tool_call_id=tool_call_id,  # Pass the tool_call_id
    )
    messages.append(tool_msg)

# Inspect the updated conversation
for message in messages:
    print(message)


[{'name': 'multiply', 'args': {'__arg1': 3, '__arg2': 12}, 'id': '1592a762-9cb7-4ce0-8283-e7c9a11c4cb4', 'type': 'tool_call'}, {'name': 'add', 'args': {'__arg1': 11, '__arg2': 49}, 'id': 'f858dff2-749c-4313-9450-6631bf79c69f', 'type': 'tool_call'}]
content='What is 3 * 12? Also, what is 11 + 49?' additional_kwargs={} response_metadata={} role='human'
content='' additional_kwargs={} response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-26T05:32:14.183382Z', 'done': True, 'done_reason': 'stop', 'total_duration': 690102125, 'load_duration': 24280334, 'prompt_eval_count': 214, 'prompt_eval_duration': 211000000, 'eval_count': 50, 'eval_duration': 453000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-29d03a80-4828-404e-92cc-4ba5b6ddb0d9-0' tool_calls=[{'name': 'multiply', 'args': {'__arg1': 3, '__arg2': 12}, 'id': '1592a762-9cb7-4ce0-8283-e7c9a11c4cb4', 'type': 'tool_call'}, {'name': 'add', 'args': {'__arg1': 11, '__arg2': 49}, 'id

In [11]:
llm_with_tools.invoke(messages)


AIMessage(content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-26T05:32:17.923489Z', 'done': True, 'done_reason': 'stop', 'total_duration': 513032000, 'load_duration': 29036250, 'prompt_eval_count': 133, 'prompt_eval_duration': 267000000, 'eval_count': 25, 'eval_duration': 215000000, 'message': Message(role='assistant', content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', images=None, tool_calls=None)}, id='run-955730b9-23ca-49ef-b7d1-53895118089b-0', usage_metadata={'input_tokens': 133, 'output_tokens': 25, 'total_tokens': 158})